In [119]:
# Author Noah Stiles
# 3/8/2022
# Threat Hunter
import paramiko
from getpass import getpass

# Create the password Prompt
thePass = getpass(prompt='Please enter your SSH Password: ')

# Host information
host = "192.168.6.71"
port = 2222
username = "noah.stiles"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

In [120]:

# Run our kraken:
kraken_command = "sudo /home/noah.stiles/kraken --folder /usr/bin --folder  /usr/sbin/ --folder /usr/local/bin --folder /sbin --folder /usr/local/sbin --folder /bin > /home/noah.stiles/kraken_test2.txt"

# Task 2
def kraken(command):
    for eachCMD in command:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)
        stdin.write(thePass + "\n") #sudo with the command

        #Establish sftp to go get the file
        sftp = ssh.open_sftp()

        # get the results
        sftp.get("/home/noah.stiles/kraken_test2.txt", "kraken_test2.txt")

        # close sftp connection
        sftp.close()

kraken(kraken_command)
# call our function


In [121]:
import re
#Open and read the file
with open('kraken_test2.txt') as ye:
    contents = (ye.readlines())
    #print(contents)
    results = []

    for line in contents:
        # Finds anything with = sign after it
        match = re.findall(r'=(\d+)' , line)
        #print(match)

        # saves the results to our list
        if len(match) > 0:
            results.append(match[0])

#print(results)


In [122]:
# Task 3
# Using PID from Kraken, use lsof to find suspcisous files
lsof_pid_cmd = ["sudo -S lsof -p"]
#Probably could make this a variable if my Kraken File was nicer to me
def lsof(command):
    for eachCMD in command:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD + str(match) + '>> lsof_outputtest2')
        stdin.write(thePass + "\n")

        sftp = ssh.open_sftp()

        # get the results
        sftp.get("/home/noah.stiles/lsof_output.txt", "lsof_outputtest2.txt")

        # close sftp connection
        sftp.close()

lsof(lsof_pid_cmd)


In [123]:
# Download the suspicous files with the given path from the lsof output
# Task 4
# Automate this?
# Instigate SFTP Session
sftp = ssh.open_sftp()
# Here are the file paths that we got from the file
sus_files_path = ["/usr/lib/x86_64-linux-gnu/libpthread-2.31.so",
            "/usr/local/bin/ls",
            "/usr/lib/x86_64-linux-gnu/ld-2.31.so",
            "/home/p.dalton/test-lk/.d.log",
            "/home/p.dalton/test-lk/.d.log",
            "/home/p.dalton/test-lk/d.pid",
    ]
# Loop through the files and download them
for each_File in sus_files_path:

    # Create a list from the filename
    x = each_File.split("/")
    # -1 plucks last element of a list
    filename = x[-1]


    # Local Path to store files
    local_path = "Suspicious_Files/" + filename

    sftp.get(each_File, local_path)


In [124]:
# Firstly check the processes running on the system
rp_command = ['ps -aux']

def in_depth_look(command):

    for eachCMD in command:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)
        stdin.write(thePass + "\n")

        # Get results from stdout

        lines = stdout.readlines()
        #print(lines)
        # Conert the list to a string
        output = ''.join(lines)
        # Header output
        sepHeader = '' + '###BEGIN ' + eachCMD + '###\n\n'
        sepFooter = '' + '###END ' + eachCMD + '###\n\n'
        # Combine the header output and footer
        cmd_output = sepHeader + output + sepFooter
        #print(cmd_output)
        # Save output to a file

        with open('running_Process.txt', 'w') as yf:
            yf.write(cmd_output)

in_depth_look(rp_command)




In [125]:
passwdFile = ['sudo -S cp /etc/passwd /home/noah.stiles/noah.stiles.passwd.txt']
def passwd(command):
    for eachCMD in command:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)
        stdin.write(thePass + "\n")

        sftp = ssh.open_sftp()

        # get the results
        sftp.get("/home/noah.stiles/noah.stiles.passwd.txt", "noah.stiles.passwd.txt")

        # close sftp connection
        #sftp.close()

passwd(passwdFile)

In [126]:

authlogFile = ["sudo -S cp /var/log/auth.log /home/noah.stiles/auth.log", "sudo -S chown naoh.stiles /home/noah.stiles/auth.log"]

def authlog(command):
    for eachCMD in command:
        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)
        stdin.write(thePass + "\n")

        sftp = ssh.open_sftp()

        # get the results
        sftp.get("/home/noah.stiles/auth.log", "auth.log")

        # close sftp connection
        sftp.close()

authlog(authlogFile)
ssh.close()